# Hierarchical ensemble

## Summed output logits

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.datasets import mnist
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from scipy import stats
from sklearn.metrics import accuracy_score
from keras import backend as K

import os
import pickle
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
SUB_REGION_SCALE = 14
NUM_HICO_LAYER = 5
NUM_TN = [10, 8, 6, 4, 2]
NUM_CONNECTION = [0, 5, 4, 3, 2]
ALPHA = 0.1

#dataset specific parameters
NUM_CLASS = 10

## Data Pre-Processing

In [ ]:
with open('/content/drive/My Drive/fyp/MNIST.pickle', 'rb') as f:
    X_train_cropped_list, y_train_one_hot, X_test_cropped_list, y_test_one_hot, coordinate_list, scale_list = pickle.load(f)

## HiCo Layer 1

In [ ]:
#build ANN model
ensemble = []
for i in range(NUM_TN[0]):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SUB_REGION_SCALE*SUB_REGION_SCALE))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append((model, i, coordinate_list[i], scale_list[i]))

In [ ]:
for i in range(NUM_TN[0]):
  ensemble[i][0].load_weights('/content/drive/My Drive/fyp/MNIST_' + str(i) + '.h5')

In [ ]:
hico_layers = []
hico_layers.append(ensemble)

## HiCo Layer 2+

In [ ]:
del hico_layers[1:]

In [ ]:
# Function to get input of layer i-1
def get_previous_layer_input(hico_layers, layer, tn, train_image):
  input = []
    
  if layer == 0:
    previous_layer_input = train_image[tn[1]]
    return previous_layer_input

  elif layer > 0:
    for i in range(NUM_CONNECTION[layer]):
      get_input = (K.function(hico_layers[layer-1][tn[1][i]][0].layers[0].input, hico_layers[layer-1][tn[1][i]][0].layers[2].output))
      input.append(get_input(get_previous_layer_input(hico_layers, layer-1, hico_layers[layer-1][tn[1][i]], train_image)))
      
    input = np.array(input)
    input = np.sum(input, axis=0)
    previous_layer_input = input
    return previous_layer_input

In [ ]:
for i in range(1, NUM_HICO_LAYER):
  print('Layer %d' %i)
  ensemble = []
  X_train_input_list = []
  X_test_input_list = []

  for j in range(NUM_TN[i]):
    # Build model of HiCo layer i
    connection = tuple(random.sample(range(len(hico_layers[i-1])), k=NUM_CONNECTION[i]))
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=NUM_CLASS))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(NUM_CLASS, activation = 'softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    ensemble.append((model, connection))
  hico_layers.append(ensemble)
  print('HICO LENGTH')
  print(len(hico_layers))

  for j in range(NUM_TN[i]):
    # Get train hidden representation from HiCo layer i-1
    X_train_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[2].output))
      X_train_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_train_cropped_list)))
    X_train_input = np.array(X_train_input)
    X_train_input = np.sum(X_train_input, axis=0)
    X_train_input_list.append(X_train_input)

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[2].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.sum(X_test_input, axis=0)
    X_test_input_list.append(X_test_input)

  #train model of HiCo layer i
  for j in range(NUM_TN[i]):
    print('Model %d' %j)
    hico_layers[i][j][0].fit(X_train_input_list[j], y_train_one_hot, validation_data=(X_test_input_list[j], y_test_one_hot), epochs=5, batch_size=128)

Layer 1
HICO LENGTH
2
Model 0
Epoch 1/5
469/469 [==============================] - 2s 3ms/step - loss: 0.6166 - accuracy: 0.8702 - val_loss: 0.1359 - val_accuracy: 0.9602
Epoch 2/5
469/469 [==============================] - 1s 3ms/step - loss: 0.1248 - accuracy: 0.9624 - val_loss: 0.1331 - val_accuracy: 0.9612
Epoch 3/5
469/469 [==============================] - 1s 2ms/step - loss: 0.1196 - accuracy: 0.9634 - val_loss: 0.1293 - val_accuracy: 0.9609
Epoch 4/5
469/469 [==============================] - 1s 2ms/step - loss: 0.1203 - accuracy: 0.9619 - val_loss: 0.1278 - val_accuracy: 0.9609
Epoch 5/5
469/469 [==============================] - 1s 2ms/step - loss: 0.1191 - accuracy: 0.9614 - val_loss: 0.1264 - val_accuracy: 0.9609
Model 1
Epoch 1/5
469/469 [==============================] - 2s 2ms/step - loss: 0.6274 - accuracy: 0.8676 - val_loss: 0.1449 - val_accuracy: 0.9557
Epoch 2/5
469/469 [==============================] - 1s 2ms/step - loss: 0.1276 - accuracy: 0.9585 - val_loss: 0.137

## Model Evaluation

In [ ]:
# Predict on test image
y_pred_list = []
for j in range(NUM_TN[0]):
  y_pred = hico_layers[0][j][0].predict(X_test_cropped_list[j])
  y_pred = y_pred * ((1 + ALPHA) - (NUM_TN[0] / max(NUM_TN)))
  y_pred_list.append(y_pred)

for i in range(1, NUM_HICO_LAYER):
  for j in range(NUM_TN[i]):

    # Get test hidden representation from HiCo layer i-1
    X_test_input = []
    for k in range(NUM_CONNECTION[i]):
      get_input = (K.function(hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[0].input, hico_layers[i-1][hico_layers[i][j][1][k]][0].layers[2].output))
      X_test_input.append(get_input(get_previous_layer_input(hico_layers, i-1, hico_layers[i-1][hico_layers[i][j][1][k]], X_test_cropped_list)))
    X_test_input = np.array(X_test_input)
    X_test_input = np.sum(X_test_input, axis=0)
    y_pred = hico_layers[i][j][0].predict(X_test_input)
    y_pred = y_pred * ((1 + ALPHA) - (NUM_TN[i] / max(NUM_TN)))
    y_pred_list.append(y_pred)


# HiCo voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.transpose(y_pred_list, (1, 0, 2))
y_pred_list = np.argmax(np.sum(y_pred_list, axis=1), axis=1)

In [ ]:
y_test = np.argmax(y_test_one_hot, axis=1)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy

0.9726